In [292]:
#Import the required libraries
import numpy as np
import pandas as pd 
import selenium
from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.common.exceptions import NoSuchElementException  

pd.options.display.max_colwidth=500
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')

In [293]:
#Connect to the webdriver
driver=webdriver.Chrome(r"C:\Users\91889\Downloads\chromedriver_win32\chromedriver.exe")
    

In [294]:
#Maximise the window
driver.maximize_window()

In [295]:
#List the sources and destinations
sources=['New Delhi','Mumbai','Kolkata','Chennai','Hyderabad','Bangalore','Pune']
destinations=['New Delhi','Mumbai','Kolkata','Chennai','Hyderabad','Bangalore','Pune']

In [467]:
#define the lists for columns of the DataFrame
origin=[]
destin=[]
flightcodes=[]
departuredate=[]
departuretime=[]
arrivaltime=[]
arrivaldate=[]
duration=[]
stops=[]
economyprice=[]
premiumprice=[]
flightinfo=[]
flightchangeplace=[]
economypriceseatsleft=[]
premiumpriceseatsleft=[]

In [469]:
for s in sources:
    for d in destinations:
        if (s==d):
            continue
        driver.get('https://www.airasia.co.in/home')
        time.sleep(0.5)
        
        #Source Input
        try:
            driver.find_elements_by_xpath("//img[@class='source-field-icon']")[0].click()
            time.sleep(0.5)
        except IndexError:
            driver.find_elements_by_xpath("//img[@class='source-field-icon']")[0].click()
            time.sleep(0.5)
            
        driver.find_element_by_xpath("//input[@class='set-outline form-control']").send_keys(s)
        try:
            driver.find_elements_by_xpath("//button[@id='list-item']")[0].click()
        except:
            continue
        
        #Destination Input
        driver.find_element_by_xpath("//input[@class='form-control']").send_keys(d)
        try:
            driver.find_elements_by_xpath("//button[@id='list-item']")[0].click()
        except:
            continue
        
        #Running a For loop for days 
        for j in range(0,30,4):
            
            #Choosing the day in the calendar
            datepicker=driver.find_element_by_xpath("//div[@class='react-google-flight-datepicker']")
            datepicker.click()
            day=driver.find_elements_by_xpath("//div[@class='day']")[j]
            driver.execute_script("arguments[0].click();",day)
            
            driver.find_element_by_xpath("//button[@class='btn-flight']").click()
            time.sleep(2.5)
           
                
            #Flight Codes    
            try:
                for i in driver.find_elements_by_xpath("//div[@class='flight-codes']"):
                    flightcodes.append(i.text)
            except NoSuchElementException:
                continue
            
            #Departure  and Arrival Time
            for n,i in enumerate(driver.find_elements_by_xpath("//span[@class='flight-time']")):
                if (n%2==0):
                    departuretime.append(i.text)
                else:
                    arrivaltime.append(i.text)
            
            
            for i in driver.find_elements_by_xpath("//span[contains(@class,'flight-code-info')]"):
                flightinfo.append(i.text)
                departuredate.append(driver.find_element_by_xpath("//div[@class='react-google-flight-datepicker']").text)
                origin.append(s)
                destin.append(d)
        
    
            for i in driver.find_elements_by_xpath("//label[@class='flight-layover-stops']"):
                flightchangeplace.append(i.text)
        
        
            for i in driver.find_elements_by_xpath("//div[@class='flight-layover-time-b2c']"):
                duration.append(i.text)
    
            for i in driver.find_elements_by_xpath("//label[@class='travel-line']"):
                try:
                    dots=i.find_elements_by_xpath(".//img[@class='stop-icon']")
                    stops.append(len(dots))
                except:
                    stops.append(0)
   
            for n,i in enumerate(driver.find_elements_by_xpath("//span[@class='price-value']")):
                try:
                    se=i.find_element_by_xpath(".//label[@class='seats-left']")
                    driver.execute_script("arguments[0];",se)
                    if (n%2==0):
                        economypriceseatsleft.append(se.text)
                    else:
                        premiumpriceseatsleft.append(se.text)
                except NoSuchElementException:
                    if (n%2==0):
                        economypriceseatsleft.append('No Status')
                    else:
                        premiumpriceseatsleft.append('No Status')
                                 
            for n,i in enumerate(driver.find_elements_by_xpath("//label[@class='discounted-price']")):
                if (n%2==0):
                    driver.execute_script("arguments[0];",i)
                    economyprice.append(i.text)
                else:
                    driver.execute_script("arguments[0];",i)
                    premiumprice.append(i.text)
        try:
            driver.find_element_by_xpath("//button[@class='volantio--dialog-frame--close--1wjmP']").click()
        except NoSuchElementException:
            pass

In [341]:
#Create a DataFrame and store the data
flightdetails=pd.DataFrame()
flightdetails['Source']=origin
flightdetails['Destination']=destin
flightdetails['Flight_Code']=flightcodes
flightdetails['Dep_Date']=departuredate
flightdetails['Dep_Time']=departuretime
flightdetails['Arrival_Time']=arrivaltime
flightdetails['Duration']=duration
flightdetails['Stops']=stops
flightdetails['Economy_Price']=economyprice
flightdetails['Premium_Price']=premiumprice
flightdetails['Flight_Info']=flightinfo
flightdetails['Flight_Changeplace']=flightchangeplace
flightdetails['SeatsLeft_EconomyPrice']=economypriceseatsleft
flightdetails['SeatsLeft_PremiumPrice']=premiumpriceseatsleft

In [465]:
flightdetails.to_csv('flightsdata.csv')